# Intro:
This notebook was authored to help demonstrate the brittleness of NSFW classifiers in the face of OOD images far away from the learnt manifold.

Specifically we focus on the false positive case that caught the media attention from here:
https://www.facebook.com/TheSeedCompanyNL/posts/2685298268399733?comment_id=2687752171487676&notif_id=1602199524375052&notif_t=feedback_reaction_generic

In [1]:
import os
import urllib
import urllib.request
# import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Rectangle# For the bounding box
from scipy.stats import describe
import itertools
from keras.preprocessing import image
from tensorflow.image import random_crop
import PIL
from PIL import Image
from tqdm.notebook import tqdm

# NSFW score model import

We use the 5-class NSFW classifier model (https://github.com/GantMan/nsfw_model).
The 5-classes are:
- drawings - safe for work drawings (including anime)
- hentai - hentai and pornographic drawings
- neutral - safe for work neutral images
- porn - pornographic images, sexual acts
- sexy - sexually explicit images, not pornography



Let us begin with the image from https://www.bbc.com/news/54467384
 

In [3]:
from nsfw_detector import predict
model_path=os.path.join(nsfw_dir,'nsfw_mobilenet2.224x224.h5')
model=predict.load_model(model_path)

In [ ]:
trial_image=os.getcwd()+'/fb_onions.png'
img = image.load_img(trial_image, target_size=(224, 224))
X=predict.classify(model,trial_image)
plt.figure()
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.barh(np.arange(5),X[trial_image].values())
plt.yticks(np.arange(5),list(X[trial_image].keys()))
plt.tight_layout()

# Experiment-1: Does this happen 'in the wild'?

In [ ]:
for im in os.listdir('./Onions'):
    trial_image='./Onions/'+im
    img = image.load_img(trial_image, target_size=(224, 224))
    X=predict.classify(model,trial_image)
    plt.figure()
    plt.subplot(121)
    plt.imshow(img)
    plt.subplot(122)
    plt.barh(np.arange(5),X[trial_image].values())
    plt.yticks(np.arange(5),list(X[trial_image].keys()))
    plt.tight_layout()

# Experiment -2: Random crops

In [ ]:
im_array=np.array(PIL.Image.open('./Onions/fb_onions.png'))
# Let us create 32 random crops 
for ind in tqdm(range(32)):
    
    im_crop=random_crop(im_array,[224,224,3])
    im = Image.fromarray(im_crop.numpy())
    trial_image=f'./onions_crop/outfile_{ind}.jpg'
    im.save(trial_image)
    X=predict.classify(model,trial_image)
    plt.figure(figsize=(12,6))
    plt.subplot(121)
    plt.imshow(im_crop)
    plt.subplot(122)
    plt.barh(np.arange(5),X[trial_image].values())
    plt.yticks(np.arange(5),list(X[trial_image].keys()))
    plt.tight_layout()
    plt.savefig(f'./onion_gif/onion_{ind}.png',dpi=300)
    plt.close()


# Convert the results to a gif:

In [ ]:
images=os.listdir('./onion_gif')
image_list = []
for file_name in images:
    image_list.append(imageio.imread('./onion_gif/'+file_name))
import imageio
imageio.mimwrite('onions.gif', image_list)